In [7]:
import requests
import polars as pl
import polars.selectors as cs

In [8]:
API_URI = "http://localhost:5432"
DATA_DIR = "../../data/favorita_dataset/subset"

def upload_json(data, endpoint):
    url = f"{API_URI}/{endpoint}"    
    post_response = requests.post(url, json=data)
    print(url, post_response.status_code)
    if post_response.status_code != 200:
        print("Error:", post_response.text)
        return {}
    return post_response.json()

    # # Get all products
    # response = requests.get(url)
    # print("get", response.status_code)
    # print(response.json())


    # # Update that product
    # # p_id = post_response.json().get("productgroups", [])[0]
    # p_id = 2
    # put_response = requests.patch(f"{url}/{p_id}", json={"pg_name": "Updated"})
    # print("update", put_response.status_code)
    # print(put_response.json())

    # response = requests.get(url)
    # print(response.status_code)
    # print(response.json())

    # assert response.status_code == 200, f"Expected status code 200, got {response.status_code}"
    
    # data = response.json()
    # assert isinstance(data, list), "Expected a list of demand predictions"
    
    # if data:
    #     assert "dp_id" in data[0], "Expected 'dp_id' in the first demand prediction"
    #     assert "dp_p_id" in data[0], "Expected 'dp_p_id' in the first demand prediction"
    #     assert "dp_s_id" in data[0], "Expected 'dp_s_id' in the first demand prediction"

def get_products():
    API_URI = "http://localhost:8000/products"
    response = requests.get(API_URI)
    print("get", response.status_code)
    if response.status_code != 200:
        print("Error:", response.text)
        return []
    
    # from response.json() get a dictionary with "p_name" as keys and "p_id" as values
    products = response.json().get("products", [])
    products_map = {product["p_name"]: product["p_id"] for product in products}
    return products_map

def get_stores():
    API_URI = "http://localhost:8000/stores"
    response = requests.get(API_URI)
    print("get", response.status_code)
    if response.status_code != 200:
        print("Error:", response.text)
        return []
    
    stores = response.json().get("stores", [])
    stores_map = {store["s_name"]: store["s_id"] for store in stores}
    return stores_map

products_map = get_products()
# stores_map = get_stores()

get 200


In [9]:
products_map

{'Pasta Vermicelli': 1,
 'Pasta Macaroni': 2,
 'Croissant': 3,
 'Pasta Spaghetti': 4,
 'Egg Pack 18': 5,
 'Donut': 6,
 'Egg Pack 30': 7}

In [2]:
print("end=")

end=


In [3]:
# 1. Upload products
table = "products"
products_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
products_ids = upload_json(products_df.to_dicts(), table).get(table, [])

http://localhost:8000/products 200


In [4]:
products_map = dict(zip(products_df["p_name"], products_ids))

In [5]:
# 2. Upload stores
table = "stores"
stores_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
stores_ids = upload_json(stores_df.to_dicts(), table).get(table, [])

http://localhost:8000/stores 200


In [6]:
stores_map = dict(zip(stores_df["s_name"], stores_ids))

In [7]:
# 3. Upload workshops
table = "workshops"
workshops_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
workshops_ids = upload_json(workshops_df.to_dicts(), table).get(table, [])

http://localhost:8000/workshops 200


In [9]:
workshops_map = dict(zip(workshops_df["w_name"], workshops_ids))

In [10]:
# 3. Upload transportlinks
table = "transportlinks"
transportlinks_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
transportlinks_df = transportlinks_df.with_columns(
    pl.col("p_name").replace_strict(products_map),
    pl.col("s_name").replace_strict(stores_map),
    pl.col("w_name").replace_strict(workshops_map)
).rename(
    {
        "p_name": "tl_p_id",
        "s_name": "tl_s_id",
        "w_name": "tl_w_id"
    }
)

transportlinks_ids = upload_json(transportlinks_df.to_dicts(), table).get(table, [])

http://localhost:8000/transportlinks 200


In [12]:
# 4. Upload procurements
table = "procurements"
procurements_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
procurements_df = procurements_df.with_columns(
    pl.col("p_name").replace_strict(products_map),
    pl.col("s_name").replace_strict(stores_map),
    cs.date().cast(pl.String)
).rename(
    {
        "p_name": "pc_p_id",
        "s_name": "pc_s_id"
    }
)

procurements_ids = upload_json(procurements_df.to_dicts(), table).get(table, [])

http://localhost:8000/procurements 200


In [13]:
# 5. Upload promotions
table = "promotions"
promotions_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
promotions_df = promotions_df.with_columns(
    pl.col("p_name").replace_strict(products_map),
    pl.col("s_name").replace_strict(stores_map),
    cs.date().cast(pl.String)
).rename(
    {
        "p_name": "pr_p_id",
        "s_name": "pr_s_id"
    }
)

promotions_ids = upload_json(promotions_df.to_dicts(), table).get(table, [])

http://localhost:8000/promotions 200


In [14]:
# 6. Upload events
table = "events"
events_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
events_ids = upload_json(events_df.to_dicts(), table).get(table, [])


http://localhost:8000/events 200


In [15]:
events_map = dict(zip(events_df["e_name"], events_ids))

In [16]:
# 7. Upload eventstores
table = "eventstores"
eventstores_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
eventstores_df = eventstores_df.with_columns(
    pl.col("e_name").replace_strict(events_map),
    pl.col("s_name").replace_strict(stores_map),
    cs.date().cast(pl.String)
).rename(
    {
        "e_name": "es_e_id",
        "s_name": "es_s_id"
    }
)

eventstores_ids = upload_json(eventstores_df.to_dicts(), table).get(table, [])

http://localhost:8000/eventstores 200


In [17]:
# 8. Upload sales
table = "sales"
sales_df = pl.read_parquet(f"{DATA_DIR}/{table}_train.parquet")
sales_df = sales_df.with_columns(
    pl.col("p_name").replace_strict(products_map),
    pl.col("s_name").replace_strict(stores_map),
    cs.date().cast(pl.String)
).rename(
    {
        "p_name": "sa_p_id",
        "s_name": "sa_s_id",
    }
)

sales_ids = upload_json(sales_df.to_dicts(), table).get(table, [])

http://localhost:8000/sales 200


In [12]:
# 9. Upload stocks
table = "stocks"
stocks_df = pl.read_parquet(f"{DATA_DIR}/{table}.parquet")
stocks_df = stocks_df.with_columns(
    pl.col("p_name").replace_strict(products_map),
    pl.col("s_name").replace_strict(stores_map),
    cs.date().cast(pl.String)
).rename(
    {
        "p_name": "sk_p_id",
        "s_name": "sk_s_id"
    }
)

stocks_ids = upload_json(stocks_df.to_dicts(), table).get(table, [])

http://localhost:8000/stocks 200
